In [ ]:
import pandas as pd
import re

# 파일 읽기 및 전처리
파일 읽기 및 디버깅을 위한 아이디값 부여<br>
kapt와 sapt(서울시공공주택정보)를 join한 테이블은 apt라고 이름붙였음

In [ ]:
apt_excel_path = '../input/kapt_sapt_병합됨.xlsx'
apt_csv_path = '../input/kapt_sapt_병합됨.csv'

건축물대장_excel_path = '../input/건축물대장_주용도필터_디자인추가.xlsx'
건축물대장_csv_path = '../input/건축물대장.csv'

In [ ]:
# apt = pd.read_excel(apt_excel_path)
# apt.to_csv(apt_csv_path, index=False)

# # 건축물대장 = pd.read_excel(건축물대장_excel_path)
# # 건축물대장.to_csv(건축물대장_csv_path, index=False)

In [ ]:
apt = pd.read_csv(apt_csv_path)
print(f"apt: {len(apt)}")

건축물대장 = pd.read_csv(건축물대장_csv_path)
print(f"건축물대장: {len(건축물대장)}")

# id 생성
apt.insert(0, 'apt_id', range(1, len(apt) + 1))
건축물대장.insert(0, '건축물대장_id', range(1, len(건축물대장) + 1))

apt = apt.rename(columns={'병합종류':'apt_병합종류'})
apt['apt_병합종류'] = apt['apt_병합종류'].replace('성공', '2번,3번')
건축물대장.insert(0, '건축물대장_병합종류', '1번')

# 법정동 주소 파싱

In [ ]:
def apt_parse_법정동주소(addr):
    if addr is None or pd.isna(addr): return(pd.NA, pd.NA, pd.NA, pd.NA)    
    addr = addr.strip()
    if not addr: return(pd.NA, pd.NA, pd.NA, pd.NA)     # 주소값이 없을 경우에 널값 리턴
    
    시도 = pd.NA
    군구 = pd.NA
    동리= pd.NA
    번지= pd.NA
    상세= pd.NA
    중복주소= pd.NA
    addr_rest = addr
    
    # 주소 파싱
    match_groups = re.match(r"(.+?시)\s*(.*)", addr)
    if match_groups:
        시도 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    match_groups = re.match(r"(.+?구)\s*(.*)", addr_rest)
    if match_groups:
        군구 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\S+)\s*(.*)", addr_rest)
    if match_groups:
        동리 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\d+(?:-\d+)?)(?:번지)?-?\s*(.*)", addr_rest)
    if match_groups:
        번지 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    if addr_rest:
        if addr_rest.startswith(','):
            중복주소 = addr_rest.lstrip(',').strip()
        else:
            상세 = addr_rest
        
    return (시도, 군구, 동리, 번지, 상세, 중복주소)

apt[
    ['법정동주소(시도)', '법정동주소(군구)', '법정동주소(동리)','법정동주소(번지)', '법정동주소(상세)', '법정동주소(중복주소)']
] = apt['법정동주소'].apply(
    lambda x: pd.Series(apt_parse_법정동주소(x))
)   

In [ ]:
def 건축물대장_parse_대지위치(addr):
    if addr is None or pd.isna(addr): return(pd.NA, pd.NA, pd.NA, pd.NA, pd.NA)
    addr = addr.strip()
    if not addr: return(pd.NA, pd.NA, pd.NA, pd.NA, pd.NA)
    
    시도 = pd.NA
    군구 = pd.NA
    동리= pd.NA
    번지= pd.NA
    상세= pd.NA
    addr_rest = addr
    
    match_groups = re.match(r"(.+?시)\s*(.*)", addr)
    if match_groups:
        시도 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    match_groups = re.match(r"(.+?구)\s*(.*)", addr_rest)
    if match_groups:
        군구 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\S+)\s*(.*)", addr_rest)
    if match_groups:
        동리 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\d+(?:-\d+)?)(?:번지)?-?\s*(.*)", addr_rest)
    if match_groups:
        번지 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    if addr_rest:
        상세 = addr_rest
        
    return (시도, 군구, 동리, 번지, 상세)

건축물대장[
    ['법정동주소(시도)', '법정동주소(군구)', '법정동주소(동리)','법정동주소(번지)', '법정동주소(상세)']
] = 건축물대장['대지위치'].apply(
    lambda addr: pd.Series(건축물대장_parse_대지위치(addr))
)   

# 도로명 주소 파싱

In [ ]:
def apt_parse_도로명주소(addr):
    if addr is None or pd.isna(addr): return(pd.NA, pd.NA, pd.NA, pd.NA)
    addr = addr.strip()
    if not addr: return(pd.NA, pd.NA, pd.NA, pd.NA)
    
    시도 = pd.NA
    군구 = pd.NA
    도로명 = pd.NA
    건물번호 = pd.NA
    중복주소 = pd.NA
    상세 = pd.NA
    addr_rest = addr
    
    match_groups = re.match(r"(.+?시)\s*(.*)", addr)
    if match_groups:
        시도 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    match_groups = re.match(r"(.+?구)\s*(.*)", addr_rest)
    if match_groups:
        군구 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\S+(?:로|길))\s*(.*)", addr_rest)
    if match_groups:
        도로명 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\d+(?:-\d+)?)\s*(.*)", addr_rest)
    if match_groups:
        건물번호 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    if addr_rest:
        if addr_rest.startswith(','):
            중복주소 = addr_rest.lstrip(',').strip()
        else:
            상세 = addr_rest
        
    return (시도, 군구, 도로명, 건물번호, 상세, 중복주소)

apt[
    ['도로명주소(시도)', '도로명주소(군구)', '도로명주소(도로명)', '도로명주소(건물번호)', '도로명주소(상세)', '도로명주소(중복주소)']
] = apt['도로명주소'].apply(
    lambda x: pd.Series(apt_parse_도로명주소(x))
)

In [ ]:
def 건축물대장_parse_도로명대지위치(addr):
    if addr is None or pd.isna(addr): return(pd.NA, pd.NA, pd.NA, pd.NA)
    addr = addr.strip()
    if not addr: return(pd.NA, pd.NA, pd.NA, pd.NA)
    
    시도 = pd.NA
    군구 = pd.NA
    도로명 = pd.NA
    건물번호 = pd.NA
    상세 = pd.NA
    addr_rest = addr
    
    match_groups = re.match(r"(.+?시)\s*(.*)", addr)
    if match_groups:
        시도 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    match_groups = re.match(r"(.+?구)\s*(.*)", addr_rest)
    if match_groups:
        군구 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\S+(?:로|길))\s*(.*)", addr_rest)
    if match_groups:
        도로명 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\d+(?:-\d+)?)\s*(.*)", addr_rest)
    if match_groups:
        건물번호 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    addr_rest = re.sub(r"\([^)]*\)", "", addr_rest).strip()
    if addr_rest:
        상세 = addr_rest
        
    return (시도, 군구, 도로명, 건물번호, 상세)

건축물대장[
    ['도로명주소(시도)', '도로명주소(군구)', '도로명주소(도로명)', '도로명주소(건물번호)', '도로명주소(상세)']
] = 건축물대장['도로명대지위치'].apply(
    lambda x: pd.Series(건축물대장_parse_도로명대지위치(x))
)

# 병합

In [ ]:
# 매칭의 기준이 되는 컬럼 리스트
법정동주소_cols = ['법정동주소(시도)','법정동주소(군구)','법정동주소(동리)','법정동주소(번지)']
도로명주소_cols = ['도로명주소(시도)', '도로명주소(군구)', '도로명주소(도로명)', '도로명주소(건물번호)']

In [ ]:
# 매칭된 데이터의 주소키는 apt의 값을 따르도록하는 함수
protected_cols = 법정동주소_cols + 도로명주소_cols
suffix = '_건축물대장'

def restore_건축물대장_cols(df):
    건축물대장_cols = [c for c in df.columns if c.endswith(suffix)]     # 출처가 건축물대장인 컬럼명 리스트
    for col in 건축물대장_cols:
        original_column_name = col.replace(suffix, '')
        if original_column_name in protected_cols:
            continue
        else:
            df[original_column_name] = df[col]  
    return df.drop(columns=건축물대장_cols)                             # 출처가 건축물대장인 주소키 컬럼 제거

In [ ]:
# apt, 건축물대장 merge
def merge_apt_with_건축물대장(
    apt_df,     # apt
    건축물대장_df, # 건축물대장
    on_list     # 매칭의 기준이 되는 컬럼 리스트
    ):
    # LEFT JOIN
    merged = apt_df.merge(
        건축물대장_df.dropna(subset=on_list, how='all'),
        on = on_list,
        how = 'left',
        suffixes = ('','_건축물대장'),  # 출처가 건축물대장인 컬럼명 뒤에 '_건축물대장'을 붙임
        indicator = True
    )
    merged = restore_건축물대장_cols(merged)    # 중복컬럼(주소키) 병합 및 suffixes 제거
    
    matched = ( # LEFT JOIN 결과물 중 양쪽 매칭에 성공한 것
    merged.loc[merged['_merge'] == 'both']
    .drop(columns=['_merge'])
    .copy()
    )
    
    unmatched_apt = ( # 매칭에 실패한 apt 데이터
        merged.loc[merged['_merge'] == 'left_only']
        .drop(columns=['_merge'])
        .copy()
    )
    
    return matched, unmatched_apt

In [ ]:
# 우선적으로는 총괄표제부만 매칭에 참여
총괄_건축물대장 = 건축물대장[건축물대장['그룹 종류'] == 0]
print(f"총괄_건축물대장 : {len(총괄_건축물대장)}")
print(f"apt : {len(apt)}")


# 법정동주소로 매칭
matched_on_법정동주소, unmatched_apt = merge_apt_with_건축물대장(apt, 총괄_건축물대장, 법정동주소_cols)
matched_on_법정동주소['매칭방식'] = '총괄,법정동'
print(f"matched_on_법정동주소 : {len(matched_on_법정동주소)}")


# 도로명주소로 매칭
matched_on_도로명주소, unmatched_apt = merge_apt_with_건축물대장(unmatched_apt, 총괄_건축물대장, 도로명주소_cols)
matched_on_도로명주소['매칭방식']='총괄,도로명'
print(f"matched_on_도로명주소 : {len(matched_on_도로명주소)}")


matched = pd.concat([matched_on_도로명주소,matched_on_법정동주소], ignore_index=True)                   # 매칭 성공한 것
merged = pd.concat([matched, unmatched_apt], ignore_index=True)    # 매칭 성공한 것 + 매칭 실패한 apt
print(f"unmatched : {len(unmatched_apt)}")
print(f"matched : {len(matched)}")
print(f"merged : {len(merged)}")

In [ ]:
# 총괄표제부를 가지지 않는 일반표제부만 매칭에 참여
일반_건축물대장 = 건축물대장[건축물대장['그룹 종류'] == 2] 


# 법정동주소로 매칭
matched_on_법정동주소, unmatched_apt = merge_apt_with_건축물대장(unmatched_apt, 일반_건축물대장, 법정동주소_cols)
matched_on_법정동주소['매칭방식'] = '일반,법정동'
print(f"matched_on_법정동주소 : {len(matched_on_법정동주소)}")


# 도로명주소로 매칭
matched_on_도로명주소, unmatched_apt = merge_apt_with_건축물대장(unmatched_apt, 일반_건축물대장, 도로명주소_cols)
matched_on_도로명주소['매칭방식']='일반,도로명'
print(f"matched_on_도로명주소 : {len(matched_on_도로명주소)}")


matched = pd.concat(
    [matched, matched_on_법정동주소, matched_on_도로명주소],
    ignore_index=True
)
merged = pd.concat(
    [matched, unmatched_apt],
    ignore_index=True
)
print(f"unmatched : {len(unmatched_apt)}")
print(f"matched: {len(matched)}")
print(f"merged : {len(merged)}")

In [ ]:
# FULL_JOIN : 매칭에 실패한 apt, 건축물대장 데이터 전부 포함
full_merged = pd.concat(
    [merged, 건축물대장]
)

full_merged = full_merged.drop_duplicates().reset_index(drop=True)  
print(len(full_merged))

In [ ]:
unnamed_columns = [col for col in full_merged.columns if "Unnamed" in col]
full_merged = full_merged.drop(columns=['나머지주소', '주소(도로명)', '주소(도로상세주소)', '매칭방식'] + unnamed_columns)

# 칼럼 병합

In [ ]:
# 병합종류 컬럼 병합
def merge_병합종류(vals1, vals2):
    # NaN, None, 공백 처리
    vals1 = '' if pd.isna(vals1) else str(vals1).strip()
    vals2 = '' if pd.isna(vals2) else str(vals2).strip()
    if vals1 == '':
        return vals2
    if vals2 == '':
        return vals1

    set1 = {v.strip() for v in vals1.split(',') if v.strip()}
    set2 = {v.strip() for v in vals2.split(',') if v.strip()}

    merged_vals = set1 | set2
    merged_vals_sorted = sorted(
        merged_vals,
        key=lambda x: int(x.replace('번', ''))
    )

    return ",".join(merged_vals_sorted)


full_merged['병합종류'] = full_merged.apply(
    lambda row: merge_병합종류(row['apt_병합종류'], row['건축물대장_병합종류']),
    axis=1
)

full_merged = full_merged.drop(
    columns=['apt_병합종류', '건축물대장_병합종류']
)

cols = ['병합종류'] + [c for c in full_merged.columns if c != '병합종류']
full_merged = full_merged[cols]

full_merged['병합종류'] = full_merged['병합종류'].replace('1번,2번,3번','성공')

In [ ]:
def _norm(val): # 값 정규화, 및 string으로
    return '' if pd.isna(val) else str(val).strip()

# input : 병합할 두 컬럼의 값 col1, col2
# output : 병합된 값, 보조값
# 한 쪽이 NaN이거나 공백일 경우, 다른 값을 병합된 값으로 함
# col1과 col2의 값이 같을 경우 col1의 값을 병합된 값으로 함
# 값이 다를 경우, col1를 병합된 값으로, col2를 보조값으로 리턴
def merge_columns(col1, col2):
    col1, col2 = _norm(col1), _norm(col2)
    merged_col=''
    sub_col=''
    
    if(col1==''): merged_col=col2
    elif(col2==''): merged_col=col1
    elif(col1==col2):
        merged_col=col1
    else:
        merged_col=col1
        sub_col=col2
    
    return merged_col, sub_col



def merge_cols_and_place(
    df: pd.DataFrame,
    blds_col: str,  # 병합할 건축물대장 컬럼명
    apt_col: str,   # 병합할 apt 컬럼명
    merged_col: str,# 병합된 결과 컬럼명
    sub_col: str    # 보조 컬럼명
):
    df[[merged_col,sub_col]] = df.apply(    # input으로 받은 건축물대장 컬럼과 apt 컬럼을 병합해 병합 컬럼과 보조 컬럼 생성
        lambda row: merge_columns(row[blds_col], row[apt_col]),
        axis=1,
        result_type='expand'
    )

    anchor_idx = df.columns.get_loc(blds_col)
    col_to_move = df.pop(merged_col)        # 병합된 컬럼과 보조 컬럼을 대응하는 건축물대장 컬럼 바로 뒤로 이동
    df.insert(anchor_idx+1,merged_col,col_to_move)
    col_to_move = df.pop(sub_col)
    df.insert(anchor_idx+2,sub_col,col_to_move)
    
    return df.drop(columns=[blds_col,apt_col]) # 병합에 쓰인 건축물대장 컬럼과 apt 컬럼 드랍한 dataframe 리턴

In [ ]:
def merge_columns_3(col1, col2, col3):
    col1, col2, col3 = _norm(col1), _norm(col2), _norm(col3)
    merged, sub1, sub2, = '','',''
    
    # 공백, 중복값은 제거
    valid_vals = []
    seen = set()
    for val in [col1,col2,col3]:
        if val != '' and val not in seen:
            valid_vals.append(val)
            seen.add(val)

    if not valid_vals:          # col1, col2, col3이 모두 유효하지 않은 경우
        return merged, sub1, sub2
    elif len(valid_vals)==1:    # 값이 모두 동일한 경우
        merged = valid_vals[0]
        return merged, sub1, sub2
    elif len(valid_vals)==2:    # 서로 다른 값이 존재할 경우
        merged, sub1 = valid_vals[0], valid_vals[1]
    elif len(valid_vals)==3:
        merged, sub1, sub2 = valid_vals[0], valid_vals[1], valid_vals[2]

    return merged, sub1, sub2



def merge_3cols_and_place(
    df: pd.DataFrame,
    col1: str,
    col2: str,
    col3: str,
    merged_col: str,
    sub_col1: str,
    sub_col2: str,
):

    df[[merged_col, sub_col1, sub_col2]] = df.apply(
        lambda row: merge_columns_3(row[col1], row[col2], row[col3]),
        axis=1,
        result_type="expand"
    )

    anchor_idx = df.columns.get_loc(col1)
    col_to_move = df.pop(merged_col)
    df.insert(anchor_idx + 1, merged_col, col_to_move)
    col_to_move = df.pop(sub_col1)
    df.insert(anchor_idx + 2, sub_col1, col_to_move)
    col_to_move = df.pop(sub_col2)
    df.insert(anchor_idx + 3, sub_col2, col_to_move)
    
    return df.drop(columns=[col1,col2,col3])

#### 단지명 컬럼 병합

In [ ]:
# 단지명은 컬럼은 한 개만
full_merged = full_merged.rename(columns={
    '단지명' : '단지명1',
    '건물명' : '단지명2'
})

full_merged = merge_cols_and_place(full_merged,'단지명1', '단지명2', '단지명', '보조_단지명')
full_merged = full_merged.drop(columns=['보조_단지명'])

#### 도로명주소 컬럼 병합

In [ ]:
# 도로명주소 컬럼은 두 개
full_merged = full_merged.rename(columns={
    '도로명주소' : '도로명주소1',
    '보조_도로명주소' : '도로명주소2',
    '도로명대지위치' : '도로명주소3'
})

def norm_도로명주소(addr : str):                    # 도로명주소 컬럼 정규화
    if not isinstance(addr, str):
        return addr
    addr = re.sub(r'\s*\([^)]*\)', '', addr)     # 괄호와 그 안의 값 제거
    addr = addr.strip()                          # 양쪽의 공백 제거
    return addr
for col in ['도로명주소1', '도로명주소2', '도로명주소3']:   
    full_merged[col] = full_merged[col].apply(norm_도로명주소)

full_merged = merge_3cols_and_place(full_merged,'도로명주소1', '도로명주소2', '도로명주소3', '도로명주소', '보조_도로명주소', '보조_도로명주소2')
full_merged = full_merged.drop(columns=['보조_도로명주소2'])      # 도로명주소 컬럼은 2개만

def merge_도로명주소(addr:str, sub_addr:str):       # '도로명주소'가 '보조_도로명주소'를 포함하는 경우, '보조_도로명주소'는 공백으로
    if (sub_addr!='') and (sub_addr in addr):
        sub_addr = ''
    return addr, sub_addr
full_merged[['도로명주소','보조_도로명주소']] = full_merged.apply(
    lambda row: merge_도로명주소(row['도로명주소'], row['보조_도로명주소']),
    axis=1,
    result_type='expand'
)

#### 사용승인일 컬럼 병합

In [ ]:
# 사용승인일은 apt의 값을 따름
# merged = merged.rename(columns={ 
#     '사용승인일_x' : '사용승인일1',
#     '사용승인일_y' : '사용승인일2',
# })

# merged = merge_cols_and_place(merged,'사용승인일1', '사용승인일2', '사용승인일', '보조_사용승인일')

#### 세대수 컬럼 병합

In [ ]:
full_merged = full_merged.rename(columns={
    '세대수' : '세대수1',
    '보조_세대수' : '세대수2',
    '세대수(세대)' : '세대수3'
})

full_merged = merge_3cols_and_place(full_merged,'세대수1', '세대수2', '세대수3', '세대수', '보조_세대수1', '보조_세대수2')

#### 건물구조 컬럼 병합

In [ ]:
# 건물구조 컬럼은 한 개만
full_merged = full_merged.rename(columns={
    '건물구조' : '건물구조1',
    '구조코드명' : '건물구조2',
})

full_merged = merge_cols_and_place(full_merged,'건물구조1', '건물구조2', '건물구조', '보조_건물구조')
full_merged=full_merged.drop(columns=['보조_건물구조'])

#### 최고층수 컬럼 병합

In [ ]:
full_merged = full_merged.rename(columns={
    '최고층수(건축물대장상)' : '최고층수(건축물대장상)1',
    '지상층수' : '최고층수(건축물대장상)2',
})

full_merged = merge_cols_and_place(full_merged,'최고층수(건축물대장상)1', '최고층수(건축물대장상)2', '최고층수(건축물대장상)', '보조_최고층수(건축물대장상)')
full_merged=full_merged.drop(columns=['보조_최고층수(건축물대장상)'])

#### 지하층수 컬럼 병합

In [ ]:
# merged = merged.rename(columns={
#     '지하층수_x' : '지하층수1',
#     '지하층수_y' : '지하층수2',
# })

# merged = merge_cols_and_place(merged,'지하층수1', '지하층수2', '지하층수', '보조_지하층수')

#### 연면적 컬럼 병합

In [ ]:
full_merged = full_merged.rename(columns={
    'k-연면적' : '연면적1',
    '연면적(㎡)' : '연면적2',
})

full_merged = merge_cols_and_place(full_merged,'연면적1', '연면적2', '연면적', '보조_연면적')

#### 건축면적 컬럼 병합

In [ ]:
full_merged = full_merged.rename(columns={
    '건축면적' : '건축면적1',
    '건축면적(㎡)' : '건축면적2',
})

full_merged = merge_cols_and_place(full_merged,'건축면적1', '건축면적2', '건축면적', '보조_건축면적')

# 디자인 추가

In [ ]:
# final_df = merged.copy()

# def color_매칭방식(val):
#     colors = {
#         '총괄,법정동': 'background-color: #04248B',# 진한 파랑
#         '총괄,도로명': 'background-color: #4D75F9',# 연한 파랑
#         '일반,법정동': 'background-color: #1D5A0C',# 진한 초록
#         '일반,도로명': 'background-color: #8AEB6F',# 연한 초록
#     }
#     return colors.get(val, '')

# 법정동_cols = ['법정동주소(시도)','법정동주소(군구)','법정동주소(동리)','법정동주소(번지)','법정동주소(상세)']
# 도로명_cols = ['도로명주소(시도)','도로명주소(군구)','도로명주소(도로명)','도로명주소(건물번호)','도로명주소(상세)']

# dup_법정동 = final_df.duplicated(subset=법정동_cols, keep=False)
# dup_도로명 = final_df.duplicated(subset=도로명_cols, keep=False)
# dup_pk = final_df['건축물대장_id'].notna() & final_df['건축물대장_id'].duplicated(keep=False)
# dup_id = final_df['apt_id'].notna() & final_df['apt_id'].duplicated(keep=False)
# def is_all_null(row, cols):
#     return row[cols].isna().all()

# def highlight_all(row):
#     styles = [''] * len(row.index)

#     # 매칭방식 컬럼 색칠
#     if '매칭방식' in row.index:
#         j = list(row.index).index('매칭방식')
#         styles[j] = color_매칭방식(row['매칭방식'])

#     # 중복인 경우 빨강
#     if dup_법정동.loc[row.name] and not is_all_null(row, 법정동_cols):
#         for i, col in enumerate(row.index):
#             if col in 법정동_cols:
#                 styles[i] = 'background-color: #ffcccc'

#     if dup_도로명.loc[row.name] and not is_all_null(row, 도로명_cols):
#         for i, col in enumerate(row.index):
#             if col in 도로명_cols:
#                 styles[i] = 'background-color: #ffcccc'
                
#     if dup_pk.loc[row.name]:
#         i = list(row.index).index('건축물대장_id')
#         styles[i] = 'background-color: #ffcccc'
        
#     if dup_id.loc[row.name]:
#         i = list(row.index).index('apt_id')
#         styles[i] = 'background-color: #ffcccc'

#     return styles

# final_styled = final_df.style.apply(highlight_all, axis=1)

In [ ]:
def highlight_merge_fail(row, main_col, sub_cols, color="#ffd900"):
    styles = [''] * len(row)

    has_fail = any(
        pd.notna(row[c]) and row[c] != ''
        for c in sub_cols
        if c in row.index
    )

    if not has_fail:
        return styles

    target_cols = [main_col] + list(sub_cols)

    for i, col in enumerate(row.index):
        if col in target_cols:
            styles[i] = f'background-color: {color}'

    return styles

In [ ]:
def highlight_주소_merge_fail(row, main_col, sub_cols, color="#ffd900"):
    styles = [''] * len(row)

    has_fail = any(
        pd.notna(row[c]) and row[c] != ''
        for c in sub_cols
        if c in row.index
    )

    if not has_fail:
        return styles

    target_cols = [main_col] + list(sub_cols)

    for i, col in enumerate(row.index):
        if col in target_cols:
            styles[i] = f'background-color: {color}'

    return styles

In [ ]:
styled = full_merged.copy().style

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='단지분류',
    sub_cols=['보조_단지분류']
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='도로명주소',
    sub_cols=['보조_도로명주소']
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='세대수',
    sub_cols=['보조_세대수1', '보조_세대수2']
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='연면적',
    sub_cols=['보조_연면적']
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='건축면적',
    sub_cols=['보조_건축면적']
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='관리방식',
    sub_cols=['보조_관리방식']
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='난방방식',
    sub_cols=['보조_난방방식']
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='경비관리방식',
    sub_cols=['보조_경비관리방식']
)

styled = styled.apply(
    highlight_merge_fail,
    axis=1,
    main_col='청소관리방식',
    sub_cols=['보조_청소관리방식']
)

# 결과

In [ ]:
full_merged.to_csv("../output/apt_건축물대장_병합.csv")
print("csv created")
styled.to_excel("../output/apt_건축물대장_병합_디자인추가.xlsx", index=False)
print("excel created")
unmatched_apt.to_excel("../output/apt_병합실패.xlsx", index=False)


# used_건축물대장_ids = (
#     matched['건축물대장_id'].unique()
# )
# unmatched_건축물대장 = 건축물대장[
#     ~건축물대장['건축물대장_id'].isin(used_건축물대장_ids)
# ].copy()
# unmatched_건축물대장.to_excel("../output/건축물대장_병합실패.xlsx", index=False)